# Universo a predecir
Me voy a quedar solamente con los departamentos a vender de Argentina 

In [1353]:
import pandas as pd

In [1354]:
dataset = pd.read_csv('dataset_cleaned.csv')

In [1355]:
dataset = dataset[dataset.l1 == 'Argentina']

In [1356]:
dataset = dataset[dataset.property_type == 'Departamento']

In [1357]:
dataset = dataset[dataset.operation_type == "Venta"]

In [1358]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 305956 entries, 148 to 952399
Data columns (total 24 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               305956 non-null  object 
 1   ad_type          305956 non-null  object 
 2   start_date       305956 non-null  object 
 3   end_date         305956 non-null  object 
 4   created_on       305956 non-null  object 
 5   lat              281166 non-null  float64
 6   lon              281143 non-null  float64
 7   l1               305956 non-null  object 
 8   l2               305956 non-null  object 
 9   l3               300920 non-null  object 
 10  l4               77301 non-null   object 
 11  rooms            284333 non-null  float64
 12  bedrooms         243918 non-null  float64
 13  bathrooms        285267 non-null  float64
 14  surface_total    160312 non-null  float64
 15  surface_covered  166755 non-null  float64
 16  price            305956 non-null  fl

In [1359]:
(dataset["start_date"] == dataset["created_on"]).value_counts()

True    305956
dtype: int64

In [1360]:
dataset.ad_type.value_counts()

Propiedad    305956
Name: ad_type, dtype: int64

In [1361]:
feature_ignore = ["price", "ad_type", "currency", "id", "created_on", "l4", "l3", "l1", "property_type", "operation_type"]

Se eliminar las siguiente features:
- price dado que vamos a utilizar el precio en dolares
- ad_type todos tienen el mismo
- currency dado que todo esta en dolares
- id es un hash no va aportar informacion
- created_on es igual a start_date siempre
- l3 cardinalidad muy alta, muy homogeneo para los populares
- l4 muchos valore nulos
- l1 son todos de argentina
- property_type son todos dptos
- operation_type son todos ventas

In [1362]:
dataset = dataset.drop(columns=feature_ignore)

In [1363]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 305956 entries, 148 to 952399
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   start_date       305956 non-null  object 
 1   end_date         305956 non-null  object 
 2   lat              281166 non-null  float64
 3   lon              281143 non-null  float64
 4   l2               305956 non-null  object 
 5   rooms            284333 non-null  float64
 6   bedrooms         243918 non-null  float64
 7   bathrooms        285267 non-null  float64
 8   surface_total    160312 non-null  float64
 9   surface_covered  166755 non-null  float64
 10  price_period     117042 non-null  object 
 11  title            305956 non-null  object 
 12  description      305953 non-null  object 
 13  dolar_price      305956 non-null  float64
dtypes: float64(8), object(6)
memory usage: 35.0+ MB


## Filtrar datos de mala calidad

##### Departamentos que tienen surface total y covered nula

In [1378]:
dataset_with_surface = dataset[dataset["surface_covered"].isna() == False]

completo las surface_total falta con surface_covered en caso que tenga, esto seria el minimo de surface_total que podria tener

In [1379]:
dataset_with_surface["surface_total"] = dataset_with_surface.apply(lambda row: row["surface_covered"] if pd.isnull(row["surface_total"]) else row["surface_total"], axis=1)

/var/folders/0f/5_5y9d3s3vbct11zf1_b27fh1rn8qs/T/ipykernel_6312/1422539656.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_with_surface["surface_total"] = dataset_with_surface.apply(lambda row: row["surface_covered"] if pd.isnull(row["surface_total"]) else row["surface_total"], axis=1)


In [1380]:
dataset_with_surface.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 166755 entries, 148 to 952399
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   start_date       166755 non-null  object 
 1   end_date         166755 non-null  object 
 2   lat              152826 non-null  float64
 3   lon              152826 non-null  float64
 4   l2               166755 non-null  object 
 5   rooms            159638 non-null  float64
 6   bedrooms         128885 non-null  float64
 7   bathrooms        163084 non-null  float64
 8   surface_total    166755 non-null  float64
 9   surface_covered  166755 non-null  float64
 10  price_period     92900 non-null   object 
 11  title            166755 non-null  object 
 12  description      166753 non-null  object 
 13  dolar_price      166755 non-null  float64
dtypes: float64(8), object(6)
memory usage: 19.1+ MB


In [1381]:
coverage_filter_surface = round((166755/ 305956) * 100, 2) 

In [1382]:
f"Dado que me quedo solo con dptos que tiene datos sobre superficie la cobertura es {coverage_filter_surface}% del dataset de ventas"


'Dado que me quedo solo con dptos que tiene datos sobre superficie la cobertura es 54.5% del dataset de ventas'

##### Departamentos sin datos de bedrooms, rooms y bathrooms

In [1383]:
dataset_with_surface = dataset_with_surface[dataset_with_surface["bedrooms"].isna() == False]
dataset_with_surface = dataset_with_surface[dataset_with_surface["rooms"].isna() == False]
dataset_with_surface = dataset_with_surface[dataset_with_surface["bathrooms"].isna() == False]
dataset_with_surface.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 124886 entries, 178 to 952399
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   start_date       124886 non-null  object 
 1   end_date         124886 non-null  object 
 2   lat              114995 non-null  float64
 3   lon              114995 non-null  float64
 4   l2               124886 non-null  object 
 5   rooms            124886 non-null  float64
 6   bedrooms         124886 non-null  float64
 7   bathrooms        124886 non-null  float64
 8   surface_total    124886 non-null  float64
 9   surface_covered  124886 non-null  float64
 10  price_period     75083 non-null   object 
 11  title            124886 non-null  object 
 12  description      124886 non-null  object 
 13  dolar_price      124886 non-null  float64
dtypes: float64(8), object(6)
memory usage: 14.3+ MB


In [1384]:
coverage_filter_rooms = round((124886/ 166755) * coverage_filter_surface, 2) 

In [1385]:
f"Dado que selecciono argentina y me quedo solo con dptos que tiene datos sobre superficie y rooms la cobertura es {coverage_filter_rooms}% del dataset original"


'Dado que selecciono argentina y me quedo solo con dptos que tiene datos sobre superficie y rooms la cobertura es 40.82% del dataset original'

##### Departamentos sin longitud y latitud

In [1388]:
dataset_venta = dataset_with_surface[dataset_with_surface["lat"].isna() != True]
dataset_venta = dataset_venta[dataset_venta.lon.isna() != True]

In [1389]:
dataset_venta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114995 entries, 178 to 952399
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   start_date       114995 non-null  object 
 1   end_date         114995 non-null  object 
 2   lat              114995 non-null  float64
 3   lon              114995 non-null  float64
 4   l2               114995 non-null  object 
 5   rooms            114995 non-null  float64
 6   bedrooms         114995 non-null  float64
 7   bathrooms        114995 non-null  float64
 8   surface_total    114995 non-null  float64
 9   surface_covered  114995 non-null  float64
 10  price_period     67949 non-null   object 
 11  title            114995 non-null  object 
 12  description      114995 non-null  object 
 13  dolar_price      114995 non-null  float64
dtypes: float64(8), object(6)
memory usage: 13.2+ MB


In [1390]:
coverage_geo = round((114995/ 124886) * coverage_filter_rooms, 2) 

In [1391]:
f"Dado que selecciono argentina y me quedo solo con dptos que tiene datos sobre superficie, rooms y (lat,log) la cobertura es {coverage_geo}% del dataset original"


'Dado que selecciono argentina y me quedo solo con dptos que tiene datos sobre superficie, rooms y (lat,log) la cobertura es 37.59% del dataset original'

## Agrego caracteristica del titulo y descripcion

In [1392]:
def has_feature(feature, row):
    title = row["title"]
    description = row["description"]
    if feature in title.lower().split(" "):
        return 1
    elif type(description) != float and feature in description.lower().split(" "):
        return 1
    else:
        return 0

In [1393]:
word_to_dig = {
    "uno": "1",
    "dos": "2",
    "tres": "3",
    "cuatro": "4",
    "cinco": "5",
    "seis": "6",
    "siete": "7",
    "ocho": "8",
    "nueve": "9",
    "diez": "10"
}
def convert_ambientes(amb):
    if len(amb) == 1:
        return amb
    elif amb in ["uno", "dos", "tres", "cuatro", "cinco", "seis", "siete", "ocho", "nueve", "diez"]:
        return word_to_dig[amb]
    else:
        
        if len(amb) > 1 and amb[len(amb)-1].isdigit():
            last_c = amb[len(amb)-1]
            return last_c
        else:
            "unknown"
        

In [1394]:
def get_before_word_ambiente(feature, row):
    title = row["title"]
    description = row["description"]
    title_words = title.lower().split(" ")
    description_words = description.lower().split(" ")
    if feature in title_words:
        index = title_words.index(feature)
        return convert_ambientes(title_words[index - 1])
    elif type(description) != float and feature in description_words:
        index = description_words.index(feature)
        return convert_ambientes(description_words[index - 1])
    else:
        return "unknown"

In [1395]:
def get_ambientes(feature, row):
    valid_ambientes = ["1", "2", "3", "4", "5", "6", "7", "8", "9"]
    amb = get_before_word_ambiente(feature, row)
    if amb in valid_ambientes:
        return amb
    else:
        return "unknown"

In [1396]:
dataset_venta["new"] = dataset_venta["title"].map(lambda title:  "nuevo" in title.lower().split(" ") or "estrenar" in title.lower().split(" "))

In [1397]:
dataset_venta["new"] = dataset_venta["new"].map(lambda x: 1 if x else 0)

In [1398]:
dataset_venta["has_security"] = dataset_venta.apply(lambda row: has_feature("seguridad",row), axis=1)

In [1399]:
dataset_venta["has_pool"] = dataset_venta.apply(lambda row: has_feature("pileta",row), axis=1)

In [1400]:

dataset_venta["has_parrilla"] = dataset_venta.apply(lambda row: has_feature("parrilla",row), axis=1)

In [1401]:
dataset_venta["has_lavadero"] = dataset_venta.apply(lambda row: has_feature("lavadero",row), axis=1)

In [1402]:
dataset_venta["has_laundry"] = dataset_venta.apply(lambda row: has_feature("laundry",row), axis=1)

In [1403]:
dataset_venta["has_garage"] = dataset_venta.apply(lambda row: has_feature("cochera",row), axis=1)

In [1404]:
dataset_venta["has_rooftop"] = dataset_venta.apply(lambda row: has_feature("terraza",row), axis=1)

In [1405]:
dataset_venta["monoambiente"] = dataset_venta.apply(lambda row: has_feature("monoambiente",row), axis=1)

In [1406]:
dataset_venta["has_balcony"] = dataset_venta.apply(lambda row: has_feature("balcon",row), axis=1)


In [1407]:
dataset_venta["ambientes"] = dataset_venta.apply(lambda row: get_ambientes("ambientes",row), axis=1)

In [1408]:
dataset_venta["front_side"] = dataset_venta.apply(lambda row: has_feature("frente",row), axis=1)

### One hot encoding de price_period

In [1409]:
dataset_venta.price_period.value_counts()

Mensual    67949
Name: price_period, dtype: int64

In [1410]:
dataset_venta["price_period_mensual"] =  dataset_venta.price_period.isna().map(lambda x: 0 if x else 1)

In [1411]:
feature_ignore_ventas = [ "title", "description", "price_period"]

In [1412]:
dataset_venta = dataset_venta.drop(columns = feature_ignore_ventas)


In [1413]:
dataset_venta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114995 entries, 178 to 952399
Data columns (total 24 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   start_date            114995 non-null  object 
 1   end_date              114995 non-null  object 
 2   lat                   114995 non-null  float64
 3   lon                   114995 non-null  float64
 4   l2                    114995 non-null  object 
 5   rooms                 114995 non-null  float64
 6   bedrooms              114995 non-null  float64
 7   bathrooms             114995 non-null  float64
 8   surface_total         114995 non-null  float64
 9   surface_covered       114995 non-null  float64
 10  dolar_price           114995 non-null  float64
 11  new                   114995 non-null  int64  
 12  has_security          114995 non-null  int64  
 13  has_pool              114995 non-null  int64  
 14  has_parrilla          114995 non-null  int64  
 15

In [1414]:
dataset_venta.describe()

,lat,lon,rooms,bedrooms,bathrooms,surface_total,surface_covered,dolar_price,new,has_security,has_pool,has_parrilla,has_lavadero,has_laundry,has_garage,has_rooftop,monoambiente,has_balcony,front_side,price_period_mensual
count,114995.000000,114995.000000,114995.000000,114995.000000,114995.000000,1.149950e+05,1.149950e+05,1.149950e+05,114995.000000,114995.000000,114995.000000,114995.000000,114995.000000,114995.000000,114995.000000,114995.000000,114995.000000,114995.000000,114995.000000,114995.000000
mean,-34.452506,-59.009350,2.824558,1.805061,1.415740,3.060812e+02,2.855461e+02,2.159199e+05,0.057594,0.123710,0.086412,0.109240,0.210592,0.028897,0.251863,0.165703,0.046880,0.112483,0.262481,0.590887
std,1.236725,1.865180,1.194953,0.949015,0.741465,4.635632e+04,4.635462e+04,3.190003e+05,0.232975,0.329252,0.280973,0.311941,0.407731,0.167518,0.434085,0.371815,0.211383,0.315961,0.439985,0.491672
min,-54.828261,-71.672311,1.000000,-1.000000,1.000000,1.000000e+00,-2.000000e+00,4.792332e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-34.617140,-58.555530,2.000000,1.000000,1.000000,4.700000e+01,4.300000e+01,8.900000e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,-34.588326,-58.451320,3.000000,2.000000,1.000000,6.500000e+01,5.800000e+01,1.370000e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,-34.544492,-58.408660,3.000000,2.000000,2.000000,9.300000e+01,8.100000e+01,2.280000e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,-24.164866,-54.050372,32.000000,32.000000,19.000000,1.111111e+07,1.111111e+07,1.664306e+07,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [1415]:
dataset_venta.head()

,start_date,end_date,lat,lon,l2,rooms,bedrooms,bathrooms,surface_total,surface_covered,...,has_parrilla,has_lavadero,has_laundry,has_garage,has_rooftop,monoambiente,has_balcony,ambientes,front_side,price_period_mensual
178,2020-08-22,2020-09-04,-34.610426,-58.414646,Capital Federal,2.0,1.0,1.0,47.0,40.0,...,0,1,0,0,1,0,0,unknown,0,1
182,2020-08-22,2020-09-04,-34.644068,-58.595643,Bs.As. G.B.A. Zona Oeste,2.0,1.0,1.0,42.0,42.0,...,0,0,0,0,0,0,0,2,0,1
183,2020-08-22,2020-09-04,-34.610426,-58.414646,Capital Federal,2.0,1.0,1.0,47.0,40.0,...,0,1,0,0,1,0,0,unknown,0,1
184,2020-08-22,2020-09-04,-34.557191,-58.480192,Capital Federal,2.0,1.0,1.0,83.0,42.0,...,1,0,0,1,1,0,0,2,0,1
185,2020-08-22,2020-09-04,-34.347729,-58.799160,Bs.As. G.B.A. Zona Norte,2.0,1.0,1.0,55.0,40.0,...,1,0,0,0,0,0,0,unknown,0,1


### Descompongo fecha

In [1416]:
dataset_venta["year"] = pd.to_datetime(dataset_venta['start_date'], format='%Y-%m-%d').map(lambda date: date.year)
dataset_venta["month"] = pd.to_datetime(dataset_venta['start_date'], format='%Y-%m-%d').map(lambda date: date.month)
dataset_venta["day"] = pd.to_datetime(dataset_venta['start_date'], format='%Y-%m-%d').map(lambda date: date.day)

In [1417]:
dataset_venta["duration"] = dataset_venta.apply(lambda row: "inf" if row['end_date'] == "9999-12-31" else (pd.to_datetime(row['end_date'], format='%Y-%m-%d') - pd.to_datetime(row['start_date'], format='%Y-%m-%d')).days , axis=1)

In [1418]:
dataset_venta = dataset_venta.drop(columns=["start_date", "end_date"])

### Me quedo con zonas mas populares

In [1419]:
dataset_venta.l2.value_counts()

Capital Federal                 67944
Bs.As. G.B.A. Zona Norte        14591
Santa Fe                        10609
Bs.As. G.B.A. Zona Sur           8375
Bs.As. G.B.A. Zona Oeste         4881
Córdoba                          2800
Buenos Aires Costa Atlántica     2538
Neuquén                          1185
Río Negro                         521
Mendoza                           467
Buenos Aires Interior             360
Tucumán                           261
Salta                             137
Entre Ríos                        102
San Juan                           78
Misiones                           61
Jujuy                              26
Chaco                              18
Chubut                             16
La Pampa                           12
San Luis                            7
Tierra Del Fuego                    6
Name: l2, dtype: int64

In [1420]:
dataset_venta["l2_cap_federal"] = dataset_venta.l2.map(lambda l2: 1 if l2 == "Capital Federal" else 0)
dataset_venta["l2_bsas_zona_norte"] = dataset_venta.l2.map(lambda l2: 1 if l2 == "Bs.As. G.B.A. Zona Norte" else 0)
dataset_venta["l2_bsas_zona_sur"] = dataset_venta.l2.map(lambda l2: 1 if l2 == "Bs.As. G.B.A. Zona Sur" else 0)
dataset_venta["l2_bsas_zona_oeste"] = dataset_venta.l2.map(lambda l2: 1 if l2 == "Bs.As. G.B.A. Zona Oeste" else 0)
dataset_venta["l2_santa_fe"] = dataset_venta.l2.map(lambda l2: 1 if l2 == "Santa Fe" else 0)
dataset_venta["l2_costa_atlantica"] = dataset_venta.l2.map(lambda l2: 1 if l2 == "Buenos Aires Costa Atlántica" else 0)

In [1421]:
dataset_venta = dataset_venta.drop(columns=["l2"])

In [1422]:
dataset_venta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114995 entries, 178 to 952399
Data columns (total 31 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   lat                   114995 non-null  float64
 1   lon                   114995 non-null  float64
 2   rooms                 114995 non-null  float64
 3   bedrooms              114995 non-null  float64
 4   bathrooms             114995 non-null  float64
 5   surface_total         114995 non-null  float64
 6   surface_covered       114995 non-null  float64
 7   dolar_price           114995 non-null  float64
 8   new                   114995 non-null  int64  
 9   has_security          114995 non-null  int64  
 10  has_pool              114995 non-null  int64  
 11  has_parrilla          114995 non-null  int64  
 12  has_lavadero          114995 non-null  int64  
 13  has_laundry           114995 non-null  int64  
 14  has_garage            114995 non-null  int64  
 15

In [1423]:
dataset_venta.to_csv('dataset_dptos_venta_with_amenities.csv', index=False)


#### Saco inmuebles extremadamente caros - outliers

In [1]:
import pandas as pd


In [2]:
dataset_venta = pd.read_csv("dataset_dptos_venta_with_amenities.csv")

In [3]:
dataset_venta["dolar_price"].quantile([.99])

0.99    1500000.0
Name: dolar_price, dtype: float64

In [4]:
dataset_venta = dataset_venta[dataset_venta["dolar_price"] < 1500000.0]

In [5]:
dataset_venta.describe()

,lat,lon,rooms,bedrooms,bathrooms,surface_total,surface_covered,dolar_price,new,has_security,...,year,month,day,duration,l2_cap_federal,l2_bsas_zona_norte,l2_bsas_zona_sur,l2_bsas_zona_oeste,l2_santa_fe,l2_costa_atlantica
count,113777.000000,113777.000000,113777.000000,113777.000000,113777.000000,1.137770e+05,1.137770e+05,1.137770e+05,113777.000000,113777.000000,...,113777.000000,113777.000000,113777.000000,113777.0,113777.000000,113777.000000,113777.00000,113777.000000,113777.000000,113777.000000
mean,-34.451158,-59.014953,2.797499,1.784886,1.391230,3.029816e+02,2.851095e+02,1.924887e+05,0.057709,0.122002,...,2020.412913,6.596263,15.786811,inf,0.586955,0.127961,0.07360,0.042900,0.093138,0.022289
std,1.242242,1.872416,1.158352,0.925278,0.695465,4.660263e+04,4.660207e+04,1.829922e+05,0.233194,0.327289,...,0.492360,3.134237,8.719953,NaN,0.492383,0.334048,0.26112,0.202632,0.290627,0.147623
min,-54.828261,-71.672311,1.000000,-1.000000,1.000000,1.000000e+00,-2.000000e+00,4.792332e+03,0.000000,0.000000,...,2020.000000,1.000000,1.000000,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
25%,-34.617460,-58.557645,2.000000,1.000000,1.000000,4.700000e+01,4.300000e+01,8.900000e+04,0.000000,0.000000,...,2020.000000,4.000000,8.000000,11.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
50%,-34.588513,-58.452158,3.000000,2.000000,1.000000,6.400000e+01,5.700000e+01,1.350000e+05,0.000000,0.000000,...,2020.000000,6.000000,16.000000,42.0,1.000000,0.000000,0.00000,0.000000,0.000000,0.000000
75%,-34.543564,-58.409284,3.000000,2.000000,2.000000,9.100000e+01,8.000000e+01,2.200000e+05,0.000000,0.000000,...,2021.000000,9.000000,23.000000,136.0,1.000000,0.000000,0.00000,0.000000,0.000000,0.000000
max,-24.164866,-54.050372,32.000000,32.000000,18.000000,1.111111e+07,1.111111e+07,1.499000e+06,1.000000,1.000000,...,2021.000000,12.000000,31.000000,inf,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000


In [6]:
dataset_venta.to_csv('dataset_dptos_venta_with_amenities.csv', index=False)
